In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

## Analyse des donnees

In [ ]:
data = pd.read_csv("final_data3.csv")
data.head(10)

In [ ]:
# les valeurs de reference. Simulation sans requete
reference_values = data.iloc[0]
# print(reference_values["FuelConsumption"])

In [ ]:
# suppression des lignes vides
final_data = data.dropna()
len(final_data)

In [ ]:
labels = [0,1,2]
# Comptage des occurrences de chaque label dans la colonne 'actionId'
counts = final_data['ActionId'].value_counts().reindex(labels, fill_value=0)

# Création de l'histogramme
plt.bar(counts.index, counts.values, tick_label=labels, edgecolor='black')

# Ajouter des labels et un titre
plt.xlabel("Type d'action")
plt.ylabel('Fréquence')
plt.title('Histogramme des occurrences des actions')

# Afficher l'histogramme
plt.show()

## Preparation des donnees X, y

Les donnees seront telles que si le trajet d'un scenario est superieur a celui du trajet de reference , le label du trajet sera ettiqueté a -1.
De meme si la consommation de carburant est superieur a celui du reference, le label sera -1